#### DNN기반 다중분류 모델 구현
- 데이터셋   : iris.csv
- 피쳐/속성  : 3개 Sepal_Length, Sepal-Width, Petal_Length,Petal.Width
- 타겟/라벨  : 1개 Setosa 외 나머지
- 학습-방법  : 지도학습 > 분류 > 다중분류(클레스 3개)
- 알고리즘   : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torchmetrics.classification import *
from torchinfo import summary
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [12]:
# 활용 패키지 버전 체크 ==> 사용자 정의 함수화 
def verCHeck():
    print(f'Pytorch.v {torch.__version__}')
    print(f'Pandas.v {pd.__version__}')
verCHeck()

Pytorch.v 2.4.1
Pandas.v 2.0.3


In [13]:
### 데이터 로딩
DATA_FILE='../data/iris.csv'
irisDF=pd.read_csv(DATA_FILE)
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [14]:
labels=dict(zip(irisDF['variety'].unique().tolist(),range(3)))
irisDF['variety']=irisDF['variety'].replace(labels)
irisDF

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [16]:
class IrisMCFModel(nn.Module):
    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()
        self.in_layer=nn.Linear(4,10)
        self.hd_layer=nn.Linear(10,5)
        self.out_layer=nn.Linear(5,3)

    #순방향 학습 진행 매서드
    def forward(self,input_data):
        # 입력층 
        y=F.relu(self.in_layer(input_data))
        # 은닉층 : 10개의 숫자 값(>=0)
        y=F.relu(self.hd_layer(y))
        # 출력층 : 5개의 숫자 값 => 다중분류 : 손실함수 CrossEntropyLoss가 내부에서  softmax진행
        return F.sigmoid(self.out_layer(y))

In [18]:
## 모델 인스턴스 생성
model=IrisMCFModel()
print(model)

IrisMCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=3, bias=True)
)


In [19]:
#모델 사용 메모리 정보 확인
summary(model,input_size=(17,4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisMCFModel                             [17, 3]                   --
├─Linear: 1-1                            [17, 10]                  50
├─Linear: 1-2                            [17, 5]                   55
├─Linear: 1-3                            [17, 3]                   18
Total params: 123
Trainable params: 123
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

[3]클레스 설계 및 정의

In [20]:
class IrisDataset(Dataset):
    
    def __init__(self,featureDF,targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
    
    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        # 피쳐와 타겟 반환
        return featureTS,targetTS

In [21]:
## 데이터셋 인스턴스 생성 
# DF에서 피쳐와 타겟 추출
featureDF=irisDF[irisDF.columns[:-1]]
targetDF=irisDF[irisDF.columns[-1:]]
# 커스텀 데이터셋 인스턴스 생성
irisDS=IrisDataset(featureDF,targetDF)
# 데이터 로더 인스턴스 생성
irisDL=DataLoader(irisDS)
for feature,label in irisDL:
    print(feature.shape,label.shape,feature,label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[0.]])


[4] 학습준비
- 학습횟수 : EPOCH 
- 배치크기 : BATCH_SIZE 
- 위치지정 : DEVICE 
- 학습률 : LR 가중치와 절편 업데이트 시 경사 하강법으로 업데이트 간격 설정 0.001~0.1

In [22]:
### 학습 진행 관련 설정
EPOCH=100
BATCH_SIZE=10
BACH_CNT=irisDF.shape[0]/BATCH_SIZE
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'
LR=0.001

In [23]:
#모델 인스턴스 
model=IrisMCFModel()

In [24]:
#DS과 DL 인스턴스
X_train,X_test,y_train,y_test=train_test_split(featureDF,targetDF,random_state=1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,random_state=1)
print(f'{X_train.shape},{X_test.shape},{X_val.shape}')
print(f'{y_train.shape},{y_test.shape},{y_val.shape}')
trainDS=IrisDataset(X_train,y_train)
valDS=IrisDataset(X_val,y_val)
testDS=IrisDataset(X_test,y_test)
# 학습용 데이터로더 인스턴스
trainDL=DataLoader(trainDS,batch_size=BATCH_SIZE)

(84, 4),(38, 4),(28, 4)
(84, 1),(38, 1),(28, 1)


In [27]:
# 최적화 인스턴스 => W,b텐서 즉, model.paramiters()전달
optimizer=optim.Adam(model.parameters(),lr=LR)
# 손실함수 인스턴스 => 분류 => 이진분류 CrossEntropyLoss 
#                            예측값은 선형식 결과 값 전달 => AF 처리 X
crossLoss=nn.CrossEntropyLoss()

[5]학습진행

In [ ]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY,SCORE_HISTORY=[[],[]],[[],[]]
CNT=len(trainDL)
for epoch in range(EPOCH):
    # 학습모드로 모델 설정
    model.train()
    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total,score_total=0,0
    for featureTS,targetTS in trainDL:
        # 학습진행
        pre_y=model(featureTS)
        # 손실계산 : nn.CrossEntropyLoss 요구사항 : 정답/타겟은 0D 또는 1D, 타입은long
        loss=crossLoss(pre_y,targetTS.reshape(-1).long())
        loss_total+=loss
        #성능평가 계산
        score=MulticlassF1Score(num_classes=3)(pre_y,targetTS.reshape(-1))
        score_total+=score
        # 최적화진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # IF 에포크당 검증 한다면
    # 모델 검증모드
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        # 평가
        pre_val=model(val_featureTS)
        # 손실
        loss_val=crossLoss(pre_val,val_targetTS.reshape(-1).long())
        # 성능평가
        score_val=MulticlassF1Score(num_classes=3)(pre_val,val_targetTS.reshape(-1))
    # 에포크당 손실값과 성능 평가값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_SIZE)
    SCORE_HISTORY[0].append(score_total/BATCH_SIZE)
    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)
    print(f'[{epoch}/{EPOCH}]\n- [TRAIN] LOSS : {LOSS_HISTORY[0][-1]} SCORE : {SCORE_HISTORY[0][-1]}')

In [37]:
 # 모델 검증모드
model.eval()
with torch.no_grad():
    # 검증 데이터셋
    test_featureTS=torch.FloatTensor(testDS.featureDF.values)
    test_targetTS=torch.FloatTensor(testDS.targetDF.values)
    # 평가
    pre_val=model(val_featureTS)
    # 손실
    loss_test=crossLoss(pre_val,val_targetTS.reshape(-1).long())
    # 성능평가
    score_test=MulticlassF1Score(num_classes=3)(pre_val,val_targetTS.reshape(-1))